In [1]:
# pip install pymysql

     -------------------------------------- 45.0/45.0 kB 221.2 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pymysql
from datetime import datetime

import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50, Xception, InceptionV3
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.xception import preprocess_input as xcep_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as incep_preprocess
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from PIL import Image, ImageTk
import threading

In [ ]:
# 初始化模型和权重
image_size = 224
resnet = ResNet50(weights=None, include_top=False, pooling='avg', input_shape=(image_size, image_size, 3))
xception = Xception(weights=None, include_top=False, pooling='avg', input_shape=(image_size, image_size, 3))
inception = InceptionV3(weights=None, include_top=False, pooling='avg', input_shape=(image_size, image_size, 3))

resnet.load_weights(r"F:\JupyterWorkSpace\BBBBBBS\models\resnet50-imagenet-finetune152.h5", by_name=True)
xception.load_weights(r"F:\JupyterWorkSpace\BBBBBBS\models\xception-imagenet-finetune116.h5", by_name=True)
inception.load_weights(r"F:\JupyterWorkSpace\BBBBBBS\models\inceptionV3-imagenet-finetune172.h5", by_name=True)

model_mix = load_model(r"F:\JupyterWorkSpace\BBBBBBS\models\mixed-model.h5")

# 分类标签
classes = ['c0安全驾驶', 'c1右手打字', 'c2右手接电话','c3左手打字','c4左手接电话',
           'c5调收音机','c6喝饮料','c7拿后面的东西','c8整理头发和化妆','c9和其他乘客说话']

In [2]:
def save_prediction_to_db(stats: dict):
    """
    将预测统计结果保存到 MySQL 数据库中。
    :param stats: dict，包含每个分类的次数，例如：
                  {
                      'c0': 3, 'c1': 0, 'c2': 1, ..., 'c9': 0
                  }
    """
    # 连接数据库
    connection = pymysql.connect(
        host='localhost',
        user='root',
        password='czy123',
        database='driver_act',
        charset='utf8mb4'
    )

    try:
        with connection.cursor() as cursor:
            # 构建 SQL 插入语句
            sql = """
                INSERT INTO prediction_stats (
                    prediction_time,
                    c0_safe_driving, c1_texting_right, c2_phone_right,
                    c3_texting_left, c4_phone_left, c5_adjust_radio,
                    c6_drinking, c7_reaching_back, c8_hair_makeup, c9_talking_to_passenger
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            values = (
                datetime.now(),
                stats.get('c0', 0), stats.get('c1', 0), stats.get('c2', 0),
                stats.get('c3', 0), stats.get('c4', 0), stats.get('c5', 0),
                stats.get('c6', 0), stats.get('c7', 0), stats.get('c8', 0), stats.get('c9', 0)
            )
            cursor.execute(sql, values)
            connection.commit()
            print("✅ 预测结果已成功写入数据库。")
    except Exception as e:
        print("❌ 写入数据库出错:", e)
    finally:
        connection.close()
        
# 测试
# prediction_stats = {
#     'c0': 3,
#     'c2': 1,
#     'c6': 2,
#     # 其余的默认为 0
# }

# save_prediction_to_db(prediction_stats)
        

### 带数据库写入的单张预测

In [16]:
def insert_prediction_to_db(driverID,class_name):
    """
    将预测类别和当前时间写入数据库。
    """
    try:
        conn = pymysql.connect(
            host='localhost',
            user='root',
            password='czy123',
            database='driver_act',
            charset='utf8mb4'
        )
        with conn.cursor() as cursor:
            sql_small = """
                INSERT INTO driver_prediction_stats (driver_id,prediction_time, predicted_class) VALUES (%s,%s, %s)
            """
            cursor.execute(sql_small, (driverID,datetime.now(), class_name))
            
#             sql_big = """
#                 INSERT INTO prediction_stats (
#                     prediction_time,driver_id,driver_name,
#                     c0_safe_driving, c1_texting_right, c2_phone_right,
#                     c3_texting_left, c4_phone_left, c5_adjust_radio,
#                     c6_drinking, c7_reaching_back, c8_hair_makeup, c9_talking_to_passenger
#                 ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
#             """
#             values = (
#                 datetime.now(),driverID,driver_name,
#                 stats.get('c0', 0), stats.get('c1', 0), stats.get('c2', 0),
#                 stats.get('c3', 0), stats.get('c4', 0), stats.get('c5', 0),
#                 stats.get('c6', 0), stats.get('c7', 0), stats.get('c8', 0), stats.get('c9', 0)
#             )
#             cursor.execute(sql_big, values)
            
        conn.commit()
        print(f"✅ 数据已写入数据库：{class_name}")
    except Exception as e:
        print(f"❌ 数据库写入失败：{e}")
    finally:
        conn.close()

def predict_single_image(driverID,image_path):
    # 加载图像
    img = load_img(image_path, target_size=(image_size, image_size))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # 提取特征
    feat_resnet = resnet.predict(resnet_preprocess(np.copy(img_array)))
    feat_xcep = xception.predict(xcep_preprocess(np.copy(img_array)))
    feat_incep = inception.predict(incep_preprocess(np.copy(img_array)))

    # 拼接特征
    bottleneck_feature = np.concatenate([feat_resnet, feat_xcep, feat_incep], axis=1)

    # 分类预测
    prediction = model_mix.predict(bottleneck_feature)
    class_idx = np.argmax(prediction[0])
    class_name = classes[class_idx]

    print("预测概率分布：", prediction[0])
    print("预测类别索引：", class_idx)
    print("预测概率：",prediction[0][class_idx])
    print("预测类别名称：", class_name)

    # 写入数据库
    insert_prediction_to_db(driverID,class_name)


In [17]:
# 测试
image_path = r"F:\BBBBBBBBBBBBBBBBB\Datas\imgs\train\c5\img_735.jpg"
driverID = 1001
predict_single_image(driverID,image_path)

1/1 [==============================] - 0s 34ms/step
预测概率分布： [0.02959111 0.06504248 0.055479   0.06771972 0.04374571 0.32144254
 0.02485283 0.31599978 0.02936878 0.04675811]
预测类别索引： 5
预测概率： 0.32144254
预测类别名称： c5调收音机
✅ 数据已写入数据库：c5调收音机
